In [62]:
import pandas as pd

In [63]:
simplification_log_path = "simplification_logs/sent_1-01-trial_parsed_log_2025-07-16_17-27-38.csv"
master_path = "master_data/all.csv" 
missing_write_path = "master_data/ignored_writing.txt" 
dest_path = "master_data/all_custom_rule.csv" 

In [64]:
# Read your CSV file
df = pd.read_csv(simplification_log_path)  # <-- Change to your filename

# Group by uid and original, join the simplifieds (remove empty, join by '. ')
agg_df = (
    df.groupby(['uid', 'original'])['simplified']
      .apply(lambda x: ' '.join(filter(None, map(str.strip, x))) )
      .reset_index()
)

In [65]:
# Rename columns to your requirements
agg_df.columns = ['uid', 'original', 'custome_rule_simplified']

# Optional: move columns to desired order (original, children, uid)
agg_df = agg_df[[ 'uid', 'original', 'custome_rule_simplified']]

In [66]:
agg_df.head()

,uid,original,custome_rule_simplified
0,1,Deshalb konnte das Fest nicht draußen stattfin...,Deshalb konnte das Fest nicht draußen stattfin...
1,1,"Weil das Wetter so schlecht war , konnte das F...","Weil das Wetter so schlecht war , konnte das F..."
2,1,das Wetter so schlecht war.,das Wetter so schlecht war. das Wetter so schl...
3,2,Deshalb kann das Fest nicht draußen stattfinden .,Deshalb kann das Fest nicht draußen stattfinde...
4,2,"Weil das Wetter so schlecht ist , kann das Fes...","Weil das Wetter so schlecht ist , kann das Fes..."


In [67]:
master_df = pd.read_csv(master_path)

In [68]:
master_df.head()

,original,simplification,original_id,simplification_id,pair_id,domain,corpus,language_level_original,language_level_simple,license,author,simple_url,complex_url,simple_title,complex_title,access_date,rater,alignment
0,"Am Mittwoch hat der Iran bekanntgegeben, dass ...",Der Iran wird teilweise aus dem Atom-Abkommen ...,296-1-1-0,296-0-1-0,296,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
1,"Die Präsidentin der EU-Kommission, Ursula von ...",Brüssel Ursula von der Leyen ist die Präsident...,628-1-1-0,628-0-1-0|628-0-1-1|628-0-1-2,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:3
2,So sollen die Treibhaus-Gase in der EU bis zum...,Bis zum Jahr 2030 soll es in der EU um 55 Proz...,628-1-1-2,628-0-1-4,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
3,In den letzten 29 Jahren schaffte man eine Ver...,"Das ist sehr viel, denn in den letzten 29 Jahr...",628-1-1-3,628-0-1-5,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
4,Für das neue Ziel bleiben weniger als 10 Jahre.,Für das neue Ziel bleiben aber weniger als 10 ...,628-1-1-4,628-0-1-6,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1


In [69]:
merged_df = master_df.merge(
    agg_df, 
    on="original", 
    how="left", 
    suffixes=('', '_customrule')
)

In [70]:
merged_df.columns

Index(['original', 'simplification', 'original_id', 'simplification_id',
       'pair_id', 'domain', 'corpus', 'language_level_original',
       'language_level_simple', 'license', 'author', 'simple_url',
       'complex_url', 'simple_title', 'complex_title', 'access_date', 'rater',
       'alignment', 'uid', 'custome_rule_simplified'],
      dtype='object')

In [71]:
#changing order of the columns

cols = list(merged_df.columns)
# Desired position: after 'original'
pos = cols.index('original') + 1
# Remove 'uid' and 'custome_rule_simplified' from their current places
cols_new = [c for c in cols if c not in ['uid', 'custome_rule_simplified']]
# Insert them in the desired order and position
for i, c in enumerate(['uid', 'custome_rule_simplified']):
    cols_new.insert(pos + i, c)
# Reorder DataFrame
merged_df = merged_df[cols_new]

In [72]:
merged_df.columns

Index(['original', 'uid', 'custome_rule_simplified', 'simplification',
       'original_id', 'simplification_id', 'pair_id', 'domain', 'corpus',
       'language_level_original', 'language_level_simple', 'license', 'author',
       'simple_url', 'complex_url', 'simple_title', 'complex_title',
       'access_date', 'rater', 'alignment'],
      dtype='object')

In [73]:
# Log out any rows from agg_df that didn't match
missing_originals = set(agg_df['original']) - set(master_df['original'])
if missing_originals:
    with open(missing_write_path, "w", encoding="utf-8") as f:
        f.write("The following 'original' values were not found in master_data/all.csv and were skipped:\n")
        for missing in missing_originals:
            f.write(missing + "\n")    
    print(f"Missing originals count: {len(missing_originals)}")

Missing originals count: 89


In [74]:
# Save the result as a new CSV
merged_df.to_csv(dest_path, index=False)
print("Done! Saved as all_custom_rule.csv")

Done! Saved as all_custom_rule.csv


In [75]:

output = pd.read_csv(dest_path)  # <-- Change to your filename
output.head()

,original,uid,custome_rule_simplified,simplification,original_id,simplification_id,pair_id,domain,corpus,language_level_original,language_level_simple,license,author,simple_url,complex_url,simple_title,complex_title,access_date,rater,alignment
0,"Am Mittwoch hat der Iran bekanntgegeben, dass ...",NaN,NaN,Der Iran wird teilweise aus dem Atom-Abkommen ...,296-1-1-0,296-0-1-0,296,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
1,"Die Präsidentin der EU-Kommission, Ursula von ...",NaN,NaN,Brüssel Ursula von der Leyen ist die Präsident...,628-1-1-0,628-0-1-0|628-0-1-1|628-0-1-2,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:3
2,So sollen die Treibhaus-Gase in der EU bis zum...,NaN,NaN,Bis zum Jahr 2030 soll es in der EU um 55 Proz...,628-1-1-2,628-0-1-4,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
3,In den letzten 29 Jahren schaffte man eine Ver...,NaN,NaN,"Das ist sehr viel, denn in den letzten 29 Jahr...",628-1-1-3,628-0-1-5,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
4,Für das neue Ziel bleiben weniger als 10 Jahre.,NaN,NaN,Für das neue Ziel bleiben aber weniger als 10 ...,628-1-1-4,628-0-1-6,628,news,News Science APA,b1,a2,save_use,APA - Austria Presse Agentur eG,https://science.apa.at/nachrichten-leicht-vers...,https://science.apa.at/nachrichten-leicht-vers...,Nachrichten leicht verständlich (in der Sprach...,Nachrichten leicht verständlich (in der Sprach...,2021-04-20,1,1:1
